In [26]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.model_selection import train_test_split



In [27]:
DATASET_NAME = 'dataset_v2'
train_df = pd.read_csv(f'../output/train_{DATASET_NAME}.csv')
train_df, valid_df = train_test_split(train_df, test_size=0.2)
train_df.head()

,id,qid1,qid2,question1,question2,is_duplicate,question1_lemma,question2_lemma,simple_ratio,partial_ratio,...,question1_punctuation_count,question2_punctuation_count,question1_hash,question2_hash,question1_degree,question2_degree,question1_degree_deviation,question2_degree_deviation,match_share,tfidf_word_match_share
319175,319177,444582,444583,What are the supply chain managementt best pra...,What are best practices in supply chain manage...,0,supply chain managementt best practices,best practice supply chain management,51,63,...,1,1,What are the supply chain managementt best pra...,What are best practices in supply chain manage...,1,1,-0.504718,-0.504718,0.600000,0.584723
364891,364894,494968,494969,Has the font size in Quora changed?,How can I increase the size of the font on Quora?,0,font size quora changed,increase size font quora,38,54,...,1,1,Has the font size in Quora changed?,How can I increase the size of the font on Quora?,1,2,-0.504718,0.495282,0.500000,0.574523
124965,124966,201694,201695,What's up with this dream?,What's up with my dreams?,0,dream,dreams,32,83,...,2,2,What's up with this dream?,What's up with my dreams?,1,1,-0.504718,-0.504718,0.500000,0.315702
307603,307605,14016,431293,What would Trump do to Syria if he were electe...,What would people learn if Donald Trump wins t...,0,would trump syria elect president,would people learn donald trump win presidenti...,40,65,...,1,1,What would Trump do to Syria if he were electe...,What would people learn if Donald Trump wins t...,3,1,1.495282,-0.504718,0.352941,0.291645
221437,221439,310792,315971,What is the best drama (genre) movie?,What do you think is the best drama film?,1,best drama genre movie,think best drama film,36,55,...,3,1,What is the best drama (genre) movie?,What do you think is the best drama film?,2,2,0.495282,0.495282,0.500000,0.398551


In [28]:
features = [
    'simple_ratio',
    'partial_ratio',
    'token_sort_ratio',
    'token_set_ratio',
    'question1_type',
    'question2_type',
    'question1_punctuation_count',
    'question2_punctuation_count',
    'match_share',
    'tfidf_word_match_share'
    ]

train_is_duplicate_df = train_df[['is_duplicate']].copy()
train_features_df = train_df[features].copy()

valid_is_duplicate_df = valid_df[['is_duplicate']].copy()
valid_features_df = valid_df[features].copy()

In [29]:
params = {}
params['objective'] = 'binary:logistic'
params['eval_metric'] = 'logloss'
params['eta'] = 0.02
params['max_depth'] = 4

d_train = xgb.DMatrix(train_features_df, label=train_is_duplicate_df)
d_valid = xgb.DMatrix(valid_features_df, label=valid_is_duplicate_df)

watchlist = [(d_train, 'train'), (d_valid, 'valid')]

bst = xgb.train(params, d_train, 400, watchlist, early_stopping_rounds=50, verbose_eval=10)

[0]	train-logloss:0.68587	valid-logloss:0.68584
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 50 rounds.
[10]	train-logloss:0.62647	valid-logloss:0.62626
[20]	train-logloss:0.58489	valid-logloss:0.58462
[30]	train-logloss:0.55422	valid-logloss:0.55393
[40]	train-logloss:0.53127	valid-logloss:0.53086
[50]	train-logloss:0.51377	valid-logloss:0.51335
[60]	train-logloss:0.50021	valid-logloss:0.49974
[70]	train-logloss:0.48946	valid-logloss:0.48902
[80]	train-logloss:0.48088	valid-logloss:0.48049
[90]	train-logloss:0.47437	valid-logloss:0.47403
[100]	train-logloss:0.46917	valid-logloss:0.46887
[110]	train-logloss:0.46514	valid-logloss:0.46491
[120]	train-logloss:0.46183	valid-logloss:0.46167
[130]	train-logloss:0.45919	valid-logloss:0.45904
[140]	train-logloss:0.45703	valid-logloss:0.45690
[150]	train-logloss:0.45530	valid-logloss:0.45523
[160]	train-logloss:0.45384	valid-logloss:0.45377
[170]	trai